In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow_probability as tfp

tfd = tfp.distributions
%env LD_LIBRARY_PATH='/home/alexander.huang/.conda/envs/tf/lib/:/home/alexander.huang/.local/lib/python3.7/site-packages/tensorrt'

env: LD_LIBRARY_PATH='/home/alexander.huang/.conda/envs/tf/lib/:/home/alexander.huang/.local/lib/python3.7/site-packages/tensorrt'


In [4]:
ion_data = pd.read_csv('../data/ionosphere.csv', header=None)
print(ion_data.info)

<bound method DataFrame.info of      0   1        2        3        4        5        6        7        8   \
0     1   0  0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708  1.00000   
1     1   0  1.00000 -0.18829  0.93035 -0.36156 -0.10868 -0.93597  1.00000   
2     1   0  1.00000 -0.03365  1.00000  0.00485  1.00000 -0.12062  0.88965   
3     1   0  1.00000 -0.45161  1.00000  1.00000  0.71216 -1.00000  0.00000   
4     1   0  1.00000 -0.02401  0.94140  0.06531  0.92106 -0.23255  0.77152   
..   ..  ..      ...      ...      ...      ...      ...      ...      ...   
346   1   0  0.83508  0.08298  0.73739 -0.14706  0.84349 -0.05567  0.90441   
347   1   0  0.95113  0.00419  0.95183 -0.02723  0.93438 -0.01920  0.94590   
348   1   0  0.94701 -0.00034  0.93207 -0.03227  0.95177 -0.03431  0.95584   
349   1   0  0.90608 -0.01657  0.98122 -0.01989  0.95691 -0.03646  0.85746   
350   1   0  0.84710  0.13533  0.73638 -0.06151  0.87873  0.08260  0.88928   

          9   ...       25     

In [5]:
def ordinal_encoder(category):
    dict = {'g': 0, 'b': 1}
    return dict[category]

print('g class:', ordinal_encoder('g'))
print('b class:', ordinal_encoder('b'))
ion_data[34] = ion_data[34].apply(ordinal_encoder)
print(ion_data[34].to_numpy())

g class: 0
b class: 1
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [6]:
# split x and y
x = ion_data.iloc[:,0:33].to_numpy()
y = ion_data.iloc[:,34].to_numpy().reshape(-1,1)

# Scaling
scaler = StandardScaler()
x = scaler.fit_transform(x)
print(x.shape)
print(y.shape)

(351, 33)
(351, 1)


In [7]:
n_examples = y.shape[0]
expectNLL = lambda y, rv_y: -rv_y.log_prob(y)/n_examples

In [8]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Contains the MCDropout layer."""


import tensorflow.compat.v2 as tf

from keras import backend
from keras.engine import base_layer
from keras.utils import control_flow_util

# isort: off
from tensorflow.python.util.tf_export import keras_export


#@keras_export("keras.layers.Dropout")
class MCDropout(base_layer.BaseRandomLayer):
    """Applies MCDropout to the input.

    The MCDropout layer randomly sets input units to 0 with a frequency of `rate`
    at each step, which helps prevent overfitting.
    Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over
    all inputs is unchanged.

    >>> tf.random.set_seed(0)
    >>> layer = tf.keras.layers.Dropout(.2, input_shape=(2,))
    >>> data = np.arange(10).reshape(5, 2).astype(np.float32)
    >>> print(data)
    [[0. 1.]
     [2. 3.]
     [4. 5.]
     [6. 7.]
     [8. 9.]]
    >>> outputs = layer(data)
    >>> print(outputs)
    tf.Tensor(
    [[ 0.    1.25]
     [ 2.5   3.75]
     [ 5.    6.25]
     [ 7.5   8.75]
     [10.    0.  ]], shape=(5, 2), dtype=float32)

    Args:
      rate: Float between 0 and 1. Fraction of the input units to drop.
      noise_shape: 1D integer tensor representing the shape of the
        binary dropout mask that will be multiplied with the input.
        For instance, if your inputs have shape
        `(batch_size, timesteps, features)` and
        you want the dropout mask to be the same for all timesteps,
        you can use `noise_shape=(batch_size, 1, features)`.
      seed: A Python integer to use as random seed.

    Call arguments:
      inputs: Input tensor (of any rank).
    """

    def __init__(self, rate=0.30, noise_shape=None, seed=None, **kwargs):
        super().__init__(seed=seed, **kwargs)
        if isinstance(rate, (int, float)) and not 0 <= rate <= 1:
            raise ValueError(
                f"Invalid value {rate} received for "
                f"`rate`, expected a value between 0 and 1."
            )
        self.rate = rate
        self.noise_shape = noise_shape
        self.seed = seed
        self.supports_masking = True

    def _get_noise_shape(self, inputs):
        # Subclasses of `Dropout` may implement `_get_noise_shape(self,
        # inputs)`, which will override `self.noise_shape`, and allows for
        # custom noise shapes with dynamically sized inputs.
        if self.noise_shape is None:
            return None

        concrete_inputs_shape = tf.shape(inputs)
        noise_shape = []
        for i, value in enumerate(self.noise_shape):
            noise_shape.append(
                concrete_inputs_shape[i] if value is None else value
            )
        return tf.convert_to_tensor(noise_shape)

    def call(self, inputs, training=None):

        output = self._random_generator.dropout(
                inputs, self.rate, noise_shape=self._get_noise_shape(inputs))
        
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = {
            "rate": self.rate,
            "noise_shape": self.noise_shape,
            "seed": self.seed,
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [9]:
class LaplaceApproximation(tf.keras.Model):

    def __init__(self,model):
        super().__init__()
        self.model = model
        self.mean = None
        self.std = None
        self.lam = None

    def fit(self,dataset,training=False):
        """
        Fit the Laplace approximation for a model. Setting the mean of the Laplace approximation to the weights of the model 
            and, using the diagonal of the Fisher matrix, set the standard deviation of the Laplace approximation.
        :param model: Model whose Fisher matrix is to be computed.
        :param dataset: Dataset which the model has been trained on, but which will not be seen in the future. 
            Formatted as (inputs, labels).
        :param samples: Number of samples to take from the dataset. More samples
            gives a better approximation of the true variance.
        """
        inputs, labels = dataset
        trainable_weights = self.model.trainable_weights
        variance = [tf.zeros_like(tensor) for tensor in trainable_weights]
        n_epochs = 3
        pb = tf.keras.utils.Progbar(inputs.shape[0])
        for e in range(n_epochs):
          for i in range(inputs.shape[0]):
              pb.update(i)
              index = i#np.random.randint(len(inputs))
              # Select an element from the dataset.
              data = inputs[index]
              target = labels[index]

              # When extracting from the array we lost a dimension so put it back.
              data = tf.expand_dims(data, axis=0)

              # Collect gradients.
              with tf.GradientTape() as tape:
                  output = self.model(data,training=training)
                  log_likelihood = output.log_prob(target)

              gradients = tape.gradient(log_likelihood, trainable_weights)

              # If the model has converged, we can assume that the current weights
              # are the mean, and each gradient we see is a deviation. The variance is
              # the average of the square of this deviation.
              variance = [var + (grad ** 2)/n_epochs for var, grad in zip(variance, gradients)]

        self.fisher_diagonal = [tensor.numpy() for tensor in variance]
        #self.fisher_diagonal = [tensor.numpy() / samples * inputs.shape[0] for tensor in variance]
        
        mins = [np.min(f) for f in self.fisher_diagonal]
        for i,x in enumerate(mins):
          if i==0 or min > x:
            min = x
        print(min)
        #print('FISHER')
        #print(fisher_diagonal)
        self.mean = self.model.get_weights()       
        

    def reset_weights(self):
        if self.mean:
            self.model.set_weights(self.mean)

    def set_std(self, weight_decay):
      self.std = [np.reciprocal(np.sqrt(x+weight_decay)) for x in self.fisher_diagonal]

    def call(self, inputs, training=False):
        e = [np.random.standard_normal(x.shape) for x in self.mean]
        sample = [m+s*e for m, s, e in zip(self.mean,self.std,e)]
        self.model.set_weights(sample)
        return self.model(inputs,training=training)

# MC Dropout Model

<span>1.</span> Train a neural network with dropout and a Bernoulli prediction for the ionosphere dataset.

In [121]:
l2_reg = tf.keras.regularizers.l2(l2=0.00001)

input = tf.keras.layers.Input(33)
h1 = tf.keras.layers.Dense(10,activation=tf.keras.activations.relu,kernel_regularizer=l2_reg,bias_regularizer=l2_reg)(input)
h1 = MCDropout(rate=0.5)(h1)
h2 = tf.keras.layers.Dense(10,activation=tf.keras.activations.relu,kernel_regularizer=l2_reg,bias_regularizer=l2_reg)(h1)
h2 = MCDropout(rate=0.5)(h2)
h3 = tf.keras.layers.Dense(10,activation=tf.keras.activations.relu,kernel_regularizer=l2_reg,bias_regularizer=l2_reg)(h2)
h3 = MCDropout(rate=0.5)(h3)
p_val = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, name="p_val")(h3)
out = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Bernoulli(probs=t,  dtype=tf.float64), name="output")(p_val)

model = tf.keras.Model(inputs=input, outputs=out, name="IonosphereMCDropout")

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.00001), loss=expectNLL)
print(model.summary())

Model: "IonosphereMCDropout"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 33)]              0         
                                                                 
 dense_21 (Dense)            (None, 10)                340       
                                                                 
 mc_dropout_21 (MCDropout)   (None, 10)                0         
                                                                 
 dense_22 (Dense)            (None, 10)                110       
                                                                 
 mc_dropout_22 (MCDropout)   (None, 10)                0         
                                                                 
 dense_23 (Dense)            (None, 10)                110       
                                                                 
 mc_dropout_23 (MCDropout)   (None, 10)        

In [122]:
model.fit(x, y, epochs=50, verbose=False)

<span>2.</span> Find the estimated aleatoric and epistemic uncertainty for all ionosphere examples using MC dropout.

In [123]:
preds = []
weight_arr = []
for _ in range(10):
    model.fit(x,y)
    y_dist = model(x)
    yhat = model.predict(x)
    preds.append(yhat)
    weights = [np.squeeze(w.numpy()) for w in model.weights]
    res = 0
    for weight in weights:
        res += np.mean(weight)
    weight_arr.append(res)

11/11 [==============================] - 0s 2ms/step


In [124]:
result = np.var(weight_arr)
print('Epistemic Uncertainty:', result)

Epistemic Uncertainty: 2.9636262827738073e-07


In [125]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
print('Aleatoric Uncertainty:', bce(y, yhat).numpy())

Aleatoric Uncertainty: 7.426827041385398


# MC Dropout with Laplace Approximation

<span>3.</span> Find the Laplace approximation of the neural network with MC dropout that was trained in answer to
question 1.

In [126]:
la = LaplaceApproximation(model)
la.fit([x,y])
la.set_std(weight_decay=1000)

347/351 [============================>.] - ETA: 0s0.0


<span>4.</span> Find the estimated aleatoric and epistemic uncertainty for all ionosphere examples using MC dropout
and the Laplace approximation.

In [130]:
preds = []
weight_arr = []
for _ in range(10):
    la = LaplaceApproximation(model)
    la.fit([x,y])
    la.set_std(weight_decay=1000)
    la_dist = la(x)
    yhat = la.predict(x)
    preds.append(yhat)
    weights = [np.squeeze(w.numpy()) for w in la.weights]
    res = 0
    for weight in weights:
        res += np.mean(weight)
    weight_arr.append(res)

11/11 [==============================] - 0s 2ms/step


In [131]:
result = np.var(weight_arr)
print('Epistemic Uncertainty:', result)

Epistemic Uncertainty: 0.0014887651228679326


In [132]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
print('Aleatoric Uncertainty:', bce(y, yhat).numpy())

Aleatoric Uncertainty: 7.514718485656673


# MC Dropout Model

<span>5.</span> Train a neural network with MC dropout and a Bernoulli prediction for the ionosphere dataset.

In [115]:
input1 = tf.keras.layers.Input(33)
h1 = tf.keras.layers.Dense(10,activation=tf.keras.activations.relu,kernel_regularizer=l2_reg,bias_regularizer=l2_reg)(input1)
h1 = MCDropout(rate=0.2)(h1)
p_val = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, name="p_val")(h1)
out1 = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Bernoulli(probs=t,  dtype=tf.float64), name="output")(p_val)

model2 = tf.keras.Model(inputs=input1, outputs=out1, name="IonosphereMCDropout2")

model2.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0000000001), loss=expectNLL)
print(model2.summary())


Model: "IonosphereMCDropout2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 33)]              0         
                                                                 
 dense_20 (Dense)            (None, 10)                340       
                                                                 
 mc_dropout_20 (MCDropout)   (None, 10)                0         
                                                                 
 p_val (Dense)               (None, 1)                 11        
                                                                 
 output (DistributionLambda)  ((None, 1),              0         
                              (None, 1))                         
                                                                 
Total params: 351
Trainable params: 351
Non-trainable params: 0
________________________________________________

In [116]:
model2.fit(x, y, epochs=50, verbose=False)

<span>6. Find the estimated aleatoric and epistemic uncertainty for all ionosphere test examples using MC
dropout.</span>

In [117]:
preds = []
weight_arr = []
for _ in range(10):
    model2.fit(x,y)
    y_dist = model2(x)
    yhat = model2.predict(x)
    preds.append(yhat)
    weights = [np.squeeze(w.numpy()) for w in model2.weights]
    res = 0
    for weight in weights:
        res += np.mean(weight)
    weight_arr.append(res)

11/11 [==============================] - 0s 2ms/step


In [118]:
result = np.var(weight_arr)
print('Epistemic Uncertainty:', result)

Epistemic Uncertainty: 1.289778403533695e-17


In [119]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
print('Aleatoric Uncertainty:', bce(y, yhat).numpy())

Aleatoric Uncertainty: 8.613361539110599
